In [1]:
import operator
import os
from collections import defaultdict, Counter
from sklearn.model_selection import train_test_split
import multiset

from utils import load_embeddings, load_projects, load_stopwords

In [2]:
projects, imapping = load_projects('.', 'java-projects - java-projects (1).csv')
labels = {k:v for k,v in zip(projects['names'], projects['labels_id'])}

In [3]:
path = "resources/java/stopwords.txt"
stopwords = load_stopwords(path)
path = "resources/en/stopwords.txt"
stopwords.update(load_stopwords(path))

In [4]:

terms_path = '../data/embeddings/terms-count/'
mapping = {v:k for k,v in imapping.items()}

In [5]:
from textblob import  Word

category_terms_count = defaultdict(lambda: Counter())
category_terms_occ = defaultdict(lambda: Counter())
text = []
lab = []
for project in labels:
    category = mapping[labels[project]]
    try:
        terms_count = load_embeddings(os.path.join(terms_path, f"{project}.vec"))
        terms = []
        for x, y in terms_count.items():
            lemma = Word(x).lemmatize()
            if lemma not in stopwords and len(x) > 1 and x not in stopwords:
                tokens = [lemma] * int(y[0])
                terms.extend(tokens)
        #terms = [[Word(x).lemmatize()] * int(y[0]) for x, y in terms_count.items() if Word(x).lemmatize() not in stopwords and len(x) > 1]
        text.append(" ".join(terms))
        lab.append(category)
    except Exception as e:
        print(e)

[Errno 2] No such file or directory: '../data/embeddings/terms-count/asm.vec'
[Errno 2] No such file or directory: '../data/embeddings/terms-count/infinispan.vec'
[Errno 2] No such file or directory: '../data/embeddings/terms-count/infer.vec'
[Errno 2] No such file or directory: '../data/embeddings/terms-count/source.vec'
[Errno 2] No such file or directory: '../data/embeddings/terms-count/org.aspectj.vec'
[Errno 2] No such file or directory: '../data/embeddings/terms-count/JGroups.vec'
[Errno 2] No such file or directory: '../data/embeddings/terms-count/checker-framework.vec'
[Errno 2] No such file or directory: '../data/embeddings/terms-count/libgdx.vec'
[Errno 2] No such file or directory: '../data/embeddings/terms-count/eclipse.platform.swt.vec'
[Errno 2] No such file or directory: '../data/embeddings/terms-count/reflectasm.vec'
[Errno 2] No such file or directory: '../data/embeddings/terms-count/flink.vec'
[Errno 2] No such file or directory: '../data/embeddings/terms-count/Smack.

In [6]:
import pandas

df = pandas.DataFrame({'text': text, 'label': lab})
df = df[~df['label'].isin(['NA', 'Miscellaneous'])]

y = df['label'].to_frame()
X = df['text']

X_train, X_test, y_train, y_test = train_test_split(
        X, y,stratify=y, test_size=0.3)

In [7]:
category_terms_occ = defaultdict(lambda: multiset.Multiset())

for label in set(y['label'].tolist()):
    corpus = X_train[y_train['label']==label].tolist()

    for sample in corpus:
        terms = sample.split(" ")
        category_terms_occ[label].update(terms)



In [8]:
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier

vectorizer = TfidfVectorizer()
vectorizer.fit(X_train)
X_t_mat = vectorizer.transform(X_train)
X_s_mat = vectorizer.transform(X_test)

In [9]:
neigh = KNeighborsClassifier(n_neighbors=7)


In [10]:
from sklearn.decomposition import PCA
pca = PCA(n_components=300)
X_t_mat_pca = pca.fit_transform(X_t_mat.todense())
X_s_mat_pca = pca.transform(X_s_mat.todense())

In [11]:
neigh.fit(X_t_mat, y_train)

<ipython-input-11-1cc042e4af8d>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  neigh.fit(X_t_mat, y_train)


KNeighborsClassifier(n_neighbors=7)

In [12]:

predictions = neigh.predict(X_s_mat)


In [13]:
confusion_matrix(y_test, predictions)

array([[ 5,  0,  1,  3,  0,  0,  1,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  1,  0,  0,  0,  0,  0,  0],
       [ 2,  0,  5,  4,  0,  0,  3,  0,  0,  0,  1],
       [ 3,  0,  1, 26,  0,  0,  2,  0,  0,  0,  0],
       [ 1,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0],
       [ 4,  0,  0,  1,  0,  2,  0,  0,  0,  0,  0],
       [ 1,  0,  1,  5,  0,  0,  4,  1,  0,  0,  0],
       [ 4,  0,  0,  4,  1,  1,  0,  2,  0,  0,  0],
       [ 3,  0,  0,  2,  0,  2,  0,  0,  3,  0,  1],
       [ 3,  0,  0,  3,  0,  1,  0,  0,  1,  4,  1],
       [ 5,  0,  0,  2,  0,  0,  0,  0,  0,  0,  5]])

In [14]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))

                        precision    recall  f1-score   support

                A-Find       0.16      0.50      0.24        10
                   CLI       1.00      0.50      0.67         2
                  Data       0.62      0.33      0.43        15
           Development       0.52      0.81      0.63        32
             Graphical       0.50      0.67      0.57         3
            Networking       0.33      0.29      0.31         7
                Parser       0.40      0.33      0.36        12
Scientific/Engineering       0.67      0.17      0.27        12
                Server       0.75      0.27      0.40        11
               Testing       1.00      0.31      0.47        13
                   Web       0.62      0.42      0.50        12

              accuracy                           0.46       129
             macro avg       0.60      0.42      0.44       129
          weighted avg       0.58      0.46      0.45       129



In [15]:
sum(pca.explained_variance_ratio_)

1.0000000000000007

In [16]:
# import autosklearn.classification
#
# automl = autosklearn.classification.AutoSklearnClassifier(
#     n_jobs=1,
#     memory_limit=140072,
#     per_run_time_limit=30,
#     tmp_folder='/tmp/autosklearn_parallel_1_example_tmp',
#     output_folder='/tmp/autosklearn_parallel_1_example_out',
#     ensemble_size=1,
#     include_preprocessors=["no_preprocessing"],
#     exclude_estimators=['liblinear_svc', 'libsvm_svc', 'mlp']
# )
# automl.fit(X_t_mat, y_train)
# y_hat = automl.predict(X_s_mat)
# print("Accuracy score", classification_report(y_test, y_hat))
#

In [17]:
# from tpot import TPOTClassifier
# tpot = TPOTClassifier(
#     generations=5,
#     population_size=50,
#     verbosity=2, random_state=42)
# tpot.fit(X_t_mat.todense(), y_train)
# print(tpot.score(X_s_mat.todense(), y_test))
# tpot.export('tpot_terms_pipeline.py')

In [48]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=.01,
                                 max_depth=3, random_state=0)
clf.fit(X_t_mat_pca, y_train)
predictions = clf.predict(X_s_mat_pca)
print(classification_report(y_test, predictions))

                        precision    recall  f1-score   support

                A-Find       0.00      0.00      0.00        10
                   CLI       0.00      0.00      0.00         2
                  Data       0.67      0.13      0.22        15
           Development       0.36      0.90      0.52        31
             Graphical       1.00      0.33      0.50         3
            Networking       0.00      0.00      0.00         8
                Parser       0.16      0.42      0.23        12
Scientific/Engineering       1.00      0.08      0.15        12
                Server       0.00      0.00      0.00        11
               Testing       0.00      0.00      0.00        13
                   Web       0.50      0.58      0.54        12

              accuracy                           0.34       129
             macro avg       0.34      0.22      0.20       129
          weighted avg       0.34      0.34      0.25       129



In [39]:
clf.fit(X_t_mat_pca, y_train['label'].tolist())

100%|██████████| 11/11 [00:00<00:00, 228.20it/s]

Scoring criteria: accuracy
--------------- model 1/11 ---------------
SGDClassifier
--------------- model 2/11 ---------------
LogisticRegression
--------------- model 3/11 ---------------
Perceptron
--------------- model 4/11 ---------------
PassiveAggressiveClassifier
--------------- model 5/11 ---------------
KMeans
--------------- model 6/11 ---------------
KNeighborsClassifier
--------------- model 7/11 ---------------
NearestCentroid
--------------- model 8/11 ---------------
RadiusNeighborsClassifier
--------------- model 9/11 ---------------
RandomForestClassifier
--------------- model 10/11 ---------------
DecisionTreeClassifier
--------------- model 11/11 ---------------
ExtraTreesClassifier
Model                          accuracy    Time/clf (s)
---------------------------  ----------  --------------
SGDClassifier                      -inf             inf
LogisticRegression                 -inf             inf
Perceptron                         -inf             inf
PassiveAg

In [37]:
clf.predict(X_t_mat_pca)

NotFittedError: This SGDClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [26]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=0.5)

In [27]:
clf.fit(X_t_mat, y_train['label'].tolist())

MultinomialNB(alpha=0.5)

In [28]:
predictions = neigh.predict(X_s_mat)
confusion_matrix(y_test, predictions)


array([[ 5,  0,  1,  3,  0,  0,  1,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  1,  0,  0,  0,  0,  0,  0],
       [ 2,  0,  5,  4,  0,  0,  3,  0,  0,  0,  1],
       [ 3,  0,  1, 26,  0,  0,  2,  0,  0,  0,  0],
       [ 1,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0],
       [ 4,  0,  0,  1,  0,  2,  0,  0,  0,  0,  0],
       [ 1,  0,  1,  5,  0,  0,  4,  1,  0,  0,  0],
       [ 4,  0,  0,  4,  1,  1,  0,  2,  0,  0,  0],
       [ 3,  0,  0,  2,  0,  2,  0,  0,  3,  0,  1],
       [ 3,  0,  0,  3,  0,  1,  0,  0,  1,  4,  1],
       [ 5,  0,  0,  2,  0,  0,  0,  0,  0,  0,  5]])

In [29]:
print(classification_report(y_test, predictions))

                        precision    recall  f1-score   support

                A-Find       0.16      0.50      0.24        10
                   CLI       1.00      0.50      0.67         2
                  Data       0.62      0.33      0.43        15
           Development       0.52      0.81      0.63        32
             Graphical       0.50      0.67      0.57         3
            Networking       0.33      0.29      0.31         7
                Parser       0.40      0.33      0.36        12
Scientific/Engineering       0.67      0.17      0.27        12
                Server       0.75      0.27      0.40        11
               Testing       1.00      0.31      0.47        13
                   Web       0.62      0.42      0.50        12

              accuracy                           0.46       129
             macro avg       0.60      0.42      0.44       129
          weighted avg       0.58      0.46      0.45       129

